In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Étape 1 : Création d'un dataset déséquilibré pour une tâche de classification binaire.
# Nous utilisons make_classification pour générer des données artificielles avec une classe majoritaire (90%) 
# et une classe minoritaire (10%).

X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [4]:
# Étape 2 : Traitement du déséquilibre des classes avec SMOTETomek.
# SMOTE est utilisé pour suréchantillonner la classe minoritaire, tandis que Tomek Links nettoie les points ambigus.
# Le résultat est un dataset d'entraînement équilibré, prêt pour l'entraînement du modèle.

from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [5]:
# Liste des modèles avec leurs paramètres et les jeux de données associés.
# - Logistic Regression : Données d'origine.
# - Random Forest : Données d'origine.
# - XGBoost : Données d'origine.
# - XGBoost avec SMOTE : Données équilibrées pour l'entraînement.
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [ ]:
# Boucle pour entraîner et évaluer chaque modèle de la liste 'models'.
# Pour chaque modèle :
# 1. Configuration des hyperparamètres avec set_params.
# 2. Entraînement sur les données d'entraînement.
# 3. Prédiction sur les données de test.
# 4. Calcul du rapport de classification (precision, recall, f1-score) avec classification_report.
# 5. Sauvegarde du rapport dans une liste pour analyse ultérieure.

reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

Importation des bibliothèques MLflow et DagsHub pour le suivi des expérimentations.
mlflow.sklearn et mlflow.xgboost ajoutent le support des modèles scikit-learn et XGBoost.
dagshub.init permet d'intégrer MLflow avec un repository DagsHub pour un suivi centralisé des expérimentations.


In [ ]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [ ]:
import dagshub
dagshub.init(repo_owner='kadermamoudou88', repo_name='Test', mlflow=True)

In [ ]:
# Configuration des variables d'environnement pour l'authentification avec le serveur MLflow hébergé sur DagsHub.
# MLFLOW_TRACKING_USERNAME : Nom d'utilisateur pour DagsHub.
# MLFLOW_TRACKING_PASSWORD : Token d'accès pour authentification sécurisée.
# MLFLOW_TRACKING_URI : URI du serveur distant MLflow pour le suivi des expérimentations.
import os 
os.environ['MLFLOW_TRACKING_USERNAME'] = 'kadermamoudou88'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'cdc80ad9f32ee76bc2993288b2b6402b2b88f646'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/kadermamoudou88/Test.mlflow'

In [ ]:
# Configuration et enregistrement des expérimentations avec MLflow
mlflow.set_experiment("Anomaly Detection")  # Définition du nom de l'expérience
mlflow.set_tracking_uri("https://dagshub.com/kadermamoudou88/Test.mlflow")  # Configuration du serveur distant

# Boucle pour journaliser chaque modèle et ses performances dans MLflow
for i, element in enumerate(models):
    model_name = element[0]  # Nom du modèle
    params = element[1]  # Hyperparamètres
    model = element[2]  # Instance du modèle
    report = reports[i]  # Rapport de classification
    
    # Démarrage d'un run MLflow
    with mlflow.start_run(run_name=model_name):  
        mlflow.log_params(params)  # Journalisation des hyperparamètres
        mlflow.log_metrics({       # Journalisation des métriques de performance
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        # Enregistrement du modèle entraîné
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")
